# Create Heatmaps for Weather Parameters

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Krasnaya Gorbatka,RU,2021-12-01 23:49:04,55.8703,41.7641,24.37,75,100,11.18
1,1,Atuona,PF,2021-12-01 23:49:04,-9.8000,-139.0333,78.58,79,57,21.27
2,2,Leh,IN,2021-12-01 23:49:04,34.1667,77.5833,18.97,50,97,3.02
3,3,Severo-Kurilsk,RU,2021-12-01 23:49:05,50.6789,156.1250,42.03,91,100,20.85
4,4,La Ronge,CA,2021-12-01 23:49:05,55.1001,-105.2842,14.25,85,90,6.91


One caveat to using gmaps: The data we use for any mapping must be either an integer or a floating-point decimal number. Let's check the data types for the columns of our DataFrame.

In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

## Create a Maximum Temperature Heatmap


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

Next, create the heatmap for the maximum temperature. The general syntax for creating a heatmap is as follows.

Google heatmaps do not plot negative numbers. If you have a maximum temperature that is less than 0 °F, then you will get an InvalidWeightException error for this line of code:

To remove the negative temperatures we can use a for loop to iterate through the max_temp and add the temperatures that are greater than 0 °F to a new list.



In [5]:
# # Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# temps = []
# for temp in max_temp:
#     # In the for loop, we're using the max() function to get the largest value between the temp and 0. 
#     #If the temp is less than 0, then 0 will be added to the list in its place. Otherwise, 
#     #the temp is added to the list
#     temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature

# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]

# 2. Assign the weights variable to some values.
    # Google heatmaps do not plot negative numbers. If you have a maximum temperature that is less than 0 °F, then you will get an InvalidWeightException error for this line of code:
max_temp = city_data_df["Max Temp"]

# 3. Assign the figure variable to the gmaps.figure() attribute.
    #Add the geographic center of Earth in the form of latitude and longitude (30.0° N and 31.0° E). Also, add a zoom level so that only one map of Earth is shown.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
                                 
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Adjust Heatmap Zoom, Intensity, and Point Radius

First, add the geographic center of Earth in the form of latitude and longitude (30.0° N and 31.0° E). Also, add a zoom level so that only one map of Earth is shown.

If you review the gmaps documentation, you may notice there is a dissipation option for creating heatmaps that can be added to the gmaps.heat_layer() attribute.

The options for this are:
1. The default option for the dissipation is "True," so we need to set our "dissipation" to "False."
2. We can add max_intensity to make each measurement have a better gradient variance.
3. We can add point_radius to make each measurement radius larger.

Also, you'll have to tweak these values until you are satisfied with the final map.

## Create a Percent Humidity Heatmap


In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Cloudiness Heatmap

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Wind Speed Heatmap


In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Get Vacation Criteria

For the app we are creating, we need to prompt the user to enter the minimum and maximum temperature ranges as floating-point decimal numbers to filter the city_data_df DataFrame.

Recall that you use the input() statement to prompt a user to enter information. To convert the input to a floating-point decimal number, wrap the input() statement with the float() method, like this: float(input()).

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Atuona,PF,2021-12-01 23:49:04,-9.8000,-139.0333,78.58,79,57,21.27
7,7,Buchanan,LR,2021-12-01 23:49:06,5.8808,-10.0467,78.82,84,30,5.41
10,10,Pedernales,DO,2021-12-01 23:49:07,18.0384,-71.7440,79.56,71,56,2.89
12,12,Aquiraz,BR,2021-12-01 23:49:08,-3.9014,-38.3911,83.21,68,85,11.68
17,17,Butaritari,KI,2021-12-01 23:49:09,3.0707,172.7902,82.22,77,96,11.01
25,25,Kupang,ID,2021-12-01 23:45:53,-10.1667,123.5833,82.98,78,40,4.61
26,26,Rikitea,PF,2021-12-01 23:46:11,-23.1203,-134.9692,76.44,73,0,17.34
27,27,Hobart,AU,2021-12-01 23:49:12,-42.8794,147.3294,88.02,58,20,1.01
34,34,Saint George,US,2021-12-01 23:47:32,37.1041,-113.5841,75.67,23,1,0.00
43,43,Hilo,US,2021-12-01 23:49:16,19.7297,-155.0900,75.61,89,75,4.61


Before moving on, determine if the preferred_cities_df DataFrame has any null values for any of the rows.

How would you determine if there are any null values for the rows in the preferred_cities_df DataFrame? (Select all that apply)



In [12]:
preferred_cities_df.count()

City_ID       164
City          164
Country       164
Date          164
Lat           164
Lng           164
Max Temp      164
Humidity      164
Cloudiness    164
Wind Speed    164
dtype: int64

In [13]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [14]:
preferred_cities_df.notnull().sum()

City_ID       164
City          164
Country       164
Date          164
Lat           164
Lng           164
Max Temp      164
Humidity      164
Cloudiness    164
Wind Speed    164
dtype: int64

Consider the following guidance:

1. Depending on the time of year and the seasons, you might have to adjust the minimum and maximum temperature to get enough cities.
2. It is a good idea to keep the number of cities to fewer than 200 to make it easier to plot the markers on the heatmap.
3. If you have some rows with null values, you'll need to drop them using the dropna() method at the end of your filtering statement when you are creating the new DataFrame.

## Get Travel Destinations

Using the coordinates from the preferred_cities_df DataFrame, find a hotel using our Google Places API and then retrieve that hotel information. Once we retrieve the hotel information, we'll need to store it so we can reference it and add the information to the pop-up marker.

Don't add the hotel information to the preferred_cities_df DataFrame because this DataFrame is our filtered DataFrame, and the customer will always filter it for each trip. We'll need to create a new DataFrame specifically for the data needed to create a heatmap and pop-up markers.

Make a copy of the preferred_cities_df DataFrame and name it hotel_df. For the hotel_df, keep the columns "City," "Country," "Max Temp," "Lat," and "Lng." Add a new column to the hotel_df DataFrame to hold the name of the hotel.

Using the latitude and longitude and specific parameters, use the Google Places Nearby Search request to retrieve a hotel and add it to the Hotel Name column.


In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,78.58,-9.8000,-139.0333,
7,Buchanan,LR,78.82,5.8808,-10.0467,
10,Pedernales,DO,79.56,18.0384,-71.7440,
12,Aquiraz,BR,83.21,-3.9014,-38.3911,
17,Butaritari,KI,82.22,3.0707,172.7902,
25,Kupang,ID,82.98,-10.1667,123.5833,
26,Rikitea,PF,76.44,-23.1203,-134.9692,
27,Hobart,AU,88.02,-42.8794,147.3294,
34,Saint George,US,75.67,37.1041,-113.5841,
43,Hilo,US,75.61,19.7297,-155.0900,


## Retrieve Hotels from a Nearby Search

The first step for retrieving hotels from a Nearby Search is to set the parameters for the search.

Set the Parameters for a Nearby Search
To find the nearest establishment to geographic coordinates, use the Google Places Nearby Search request. First, navigate to the Nearby Search requests page.


Specific parameters include:
1. Our API key
2. A location, which will be the latitude and longitude
3. A radius for the search. The radius can be up to 50,000 meters or approximately 31 miles. This distance is much too great for finding hotels, so we'll keep our search to 5,000 meters.
4. If we use the rankby=distance for a parameter, we need to add one or more of the three parameters above. We won't use the rankby=distance because we will be search based on the radius from a given latitude and longitude.
5. We can add optional parameters such as a keyword, a type of business, or the name of a business.

Next, we'll need to look at the documentation on the Python Requests Library (Links to an external site.) provided earlier. The section "Passing Parameters In URLs" states that we can add the parameters as a dictionary of strings, using the params keyword argument requests.get('base URL', params=parameters).


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

Next, add the coordinates for the location parameter for each city, which need to be pulled from the Lat and Lng columns of the hotel_df DataFrame. We'll need to iterate through the columns of the DataFrame and add the coordinates to the params dictionary before making the request and retrieving the JSON data. However, before iterating through the hotel_df DataFrame, we need to know what the JSON data looks like from a search. Let's practice using a fixed latitude and longitude so we understand where to get the hotel name.

## Iterate Through hotel_df DataFrame

We can use the iterrows() function to perform the iteration; however, we need to provide the index and the row in the for loop using this syntax: for index, row in df.iterrows().

In [19]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,78.58,-9.8000,-139.0333,Villa Enata
7,Buchanan,LR,78.82,5.8808,-10.0467,Kamaneahn Hotel Inc
10,Pedernales,DO,79.56,18.0384,-71.7440,Hotel Costa Marina
12,Aquiraz,BR,83.21,-3.9014,-38.3911,Pousada Rua Camurupim
17,Butaritari,KI,82.22,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...
537,Lemoore,US,75.20,36.3008,-119.7829,Travelodge by Wyndham Lemoore Near Naval Air S...
542,Bria,CF,77.63,6.5423,21.9863,WFP Guesthouse
552,Cururupu,BR,77.43,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
556,Puro,PH,78.60,12.4841,123.3827,


## Create a Maximum Temperature Heatmap from a Hotel DataFrame
Before we add pop-up markers with hotel information, let's create a heatmap using the maximum temperature from our hotel_df DataFrame by reusing the code and changing the DataFrame name.

If we refer to the gmaps documentation on how to add markers (Links to an external site.), the syntax is markers = gmaps.marker_layer(marker_locations), where the marker_locations are latitudes and longitudes.

Let's add the markers for each city on top of the heatmap. Edit the code we used to create the heatmap to look like the following:

In [23]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

Now we can add a pop-up marker for each city that displays the hotel name, city name, country, and maximum temperature.

From the gmaps documentation on how to add markers (Links to an external site.), we need to add an info_box_template that has the following syntax.

info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

Next, add the data to the code by iterating through the hotel_df DataFrame using the iterrows() function. Then add the information to the gmaps.marker_layer() attribute with the locations.

According to the documentation (Links to an external site.), we can use this example to create the data for the info_box_template:

From the documentation, we can take this code:

plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]

Then edit this code by replacing "plant_info" with "hotel_info," and then use the iterrows() function to get the index and data in the row to add to the marker.

Add the following code below to the info_box_template code and run the cell.

In [28]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


Next, in the code we used to create the heatmap with markers, add info_box_content=hotel_info to the gmaps.marker_layer() attribute with the locations. Our final cell should look like the following.

In [30]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))